# Examples

In this notebook, we showcase a few simple examples of how to interact with `libgravix2`'s API. First, we have to import the Python binding...

In [1]:
from src import gravix2

... and a few more libraries:

In [2]:
import matplotlib.pyplot as plt
import numpy as np

# https://pypi.org/project/nbfigtulz/
import nbfigtulz as ftl

For plotting we use [`nbfigtulz`](https://pypi.org/project/nbfigtulz/) and save images into a temporary directory:

In [3]:
import tempfile

img_dir = tempfile.TemporaryDirectory()
ftl.config['img_dir'] = img_dir.name

Now, we can load the library:

In [4]:
grvx = gravix2.load_library('libs/libgravix2.so')
grvx.config

Config(pot_type='2D', n_pot=None, trajectory_size=100, int_steps=10, min_dist=0.02, composition_scheme='p8s15')

Above, we already printed the static configurations of the library. Another static information is the escape velocity $v_\pi$:

In [5]:
grvx.v_esc

4.2919398191075855

This is a handy unit to quantify velocities. For example, we can query the orbital period, i.e., the number of time steps it takes for a missile to crash into the very same planet it was launched from, when started with an initial velocity $v_0=v_\pi/2$ or $v_0=2v_\pi$:

In [6]:
for p in [-2, -3, -4, -5]:
    h = 10**p
    t1 = grvx.estimate_orb_period(v0=.5 * grvx.v_esc, h=h)
    t2 = grvx.estimate_orb_period(v0=2. * grvx.v_esc, h=h)
    print(f'h=1e{p}: {t1 * h:f} | {t2 * h:f}')

h=1e-2: 0.010182 | 0.291101
h=1e-3: 0.009817 | 0.082222
h=1e-4: 0.009771 | 0.082142
h=1e-5: 0.009766 | 0.082132


There are several things to note here:
 - The values of the escape velocity and the orbital period, as returned by `grvx.v_esc` and `grvx.estimate_orb_periode()`, have only a straightforward intepretation for **isolated planets**. If more than one planet is present, missiles do not (necessarily) move on [great circles](https://en.wikipedia.org/wiki/Great_circle). Therefore, the true number of timesteps to crash into the planet can vary significantly and can even be infinite when the missile crashes into (one of) the other planet(s) first. Still, both quantities are useful for normalizing velocities and number of timesteps.
 - The results of missile propagation simulation depend on the step size `h`. In general, smaller values of `h` give more precise results but also require longer integrations. This is the reason why we scaled the orbital periods `t1` and `t2` with `h` which compensates this effect.
 - Ideally, the product `t1 * h` and `t2 * h` should not depend on our choice of `h`. However, in reality we see such a dependence. Finding a _good_ value for `h` is important but depends on several factors: For small velocities, `h` can be set to larger values but defining what sufficiently _small_ means is highly subjective and depends on the given context. As shown above, `h=1e-2` might be reasonable approximation for $v_0 = v_\pi / 2$ but arguably a poor approximation for $v_0 = 2 v_\pi$ (for the 3D force field this situation is flipped). Furthermore, most times we do not know the true velocity but have to use the simulation to approximate it. That is, chosing `h` poorly also spoils our judgement whether or not `h` was set to a sufficiently small value.
 - Finally, be aware that changing `h` during propagation will most likely destroy the symplectic and symmetric properties of the integrator.

Let's create a missile🚀

In [7]:
missiles = grvx.new_missiles(2)
m = missiles[0]

... and write a handy helper to propagate the missile multiple times.

In [8]:
def shoot(missile, *, n=1, h=1e-3):
    x = np.empty((0, 3))
    v = np.empty((0, 3))
    
    premature = False
    while n > 0 and not premature:
        n -= 1
        premature = missile.propagate(planets, h=h)
        if premature:
            print('!!! PREMATURE !!!')
        
        x = np.concatenate((x, missile.trajectory.x), axis=0)
        v = np.concatenate((v, missile.trajectory.v), axis=0)
    
    return x, v

## Escape velocity

Time to create a planet and to accelerate the missile to escape velocity 🌎🚀

In [9]:
planets = grvx.new_planets([(0., 0.)])
m.launch(planets=planets, planet_idx=0, v=grvx.v_esc, psi=np.pi/2.)

n = 50 if grvx.config.pot_type == '2D' else 200
x, v = shoot(m, n=n)
x.shape, v.shape

!!! PREMATURE !!!


((3521, 3), (3521, 3))

Bamm! We tried to propagate the missile for

In [10]:
n * grvx.config.trajectory_size

5000

time steps. Apparently, the missile came back after

In [11]:
x.shape[0]

3521

time steps. Let's visualize this:

In [12]:
@ftl.with_context
def make_fig(x, label, *, file_name):
    fig, ax = plt.subplots()
    ax.set_xlabel('Time step')
    ax.set_ylabel(label)
    ax.grid()
    ax.plot(np.rad2deg(x), '.', alpha=.2)
        
    return ftl.save_fig(fig, file_name)
        
        
ftl.img_grid([
    make_fig(np.arcsin(x[:, 2]), 'Latitude (deg.)', file_name='vesc_lat'),
    make_fig(np.arctan2(x[:, 0], x[:, 1]), 'Longitude (deg.)', file_name='vesc_lon'),
])

Obviously, we tried to balance a pencil on its tip: we launched the missile at the equator of our universe westwards. The velocity was chosen such that the missile barely reaches the exact opposite point of our universe at $(\phi=0^\circ, \lambda=\pm180^\circ)$. Here, its velocity is very close to zero but numerical errors accumulate and the missile eventially leaves the unstable fix point. (Note that our universe is a sphere and that at the same latitude, the longitudinals $\lambda = -179^\circ$ and $\lambda= +179^\circ$ are only separated by $2^\circ$.)

Let's fire another missile northwards but this time with twice the escape velocity...

In [13]:
m.launch(planets=planets, planet_idx=0, v=2. * grvx.v_esc, psi=0.)

x, v = shoot(m, n=1)
x.shape, v.shape

!!! PREMATURE !!!


((83, 3), (83, 3))

... and visualize the trajectory

In [14]:
ftl.img_grid([
    make_fig(np.arcsin(x[:, 2]), 'Latitude (deg.)', file_name='vesc2_lat'),
    make_fig(np.arctan2(x[:, 0], x[:, 1]), 'Longitude (deg.)', file_name='vesc2_lon'),
])

Phew, this missile was fast! The trajectory almost looks as if no planet was attracting the missile at all! Although not significanty contributing to the acceleration of the missile, the very last point shows the sudden stop of the propagation at the surface of the planet: That is, only $k-1$ points of a trajectory of length $k$ are equidistantly distributed in time; if the missile collides with a planet the temporal distance between the point $(k-1)$-th and $k$-th point is shorter.🤓

## Two planets

Previously, we studied isolated planets. Let's add a second planet 🌎🚀🌎. Note that we can still use $v_\pi$ as a natural unit to normalize velocities:

In [15]:
planets = grvx.new_planets([
    (np.deg2rad(10.), np.deg2rad(20.)),
    (np.deg2rad(-20.), np.deg2rad(-30)),
])

v0 = (.9 if grvx.config.pot_type == '2D' else 1.) * grvx.v_esc
m.launch(planets=planets, planet_idx=0, v=v0, psi=np.deg2rad(90.))

x, v = shoot(m, n=9)
x.shape, v.shape

((900, 3), (900, 3))

In order to test the symmetric property of the integrator we initialize a second missile at the last point of the first missile and invert the momentum 🌎🚀🚀🌎

In [16]:
m2 = missiles[1]
m2.set(pos=x[-1], orientation=-v[-1], v=np.sqrt(np.sum(v[-1] ** 2)))

x2, v2 = shoot(m2, n=9)
x2.shape, v2.shape

((900, 3), (900, 3))

Below, we show both trajectories:

In [17]:
@ftl.with_context
def make_fig(x, planets, *, file_name):
    lat = np.arcsin(x[:, 2])
    lon = np.arctan2(x[:, 0], x[:, 1])

    fig, ax = plt.subplots()
    ax.plot(np.rad2deg(lon), np.rad2deg(lat))

    ax.grid()
    ax.set_xlabel('Longitude (deg.)')
    ax.set_ylabel('Latitude (deg.)')

    for lat, lon in planets:
        ax.plot(np.rad2deg(lon), np.rad2deg(lat), 'o', c='black', alpha=.5)
        
    return ftl.save_fig(fig, file_name)
        
        
ftl.img_grid([
    make_fig(x, planets.planet_pos, file_name='p2'),
    make_fig(x2, planets.planet_pos, file_name='p2_reverse'),
])

,


The trajectories look identical and thus proofs the temporal symmetry. In fact, the error is very small...

In [18]:
@ftl.with_context
def make_fig(x1, x2):
    lat1 = np.arcsin(x1[:, 2])
    lon1 = np.arctan2(x1[:, 0], x1[:, 1])
    
    lat2 = np.arcsin(x2[:, 2])
    lon2 = np.arctan2(x2[:, 0], x2[:, 1])

    fig, ax = plt.subplots()
    
    ax.grid()
    ax.plot(np.rad2deg(lat1 - lat2[::-1]), label=r'$\Delta \phi$')
    ax.plot(np.rad2deg(lon1 - lon2[::-1]), label=r'$\Delta \lambda$')
    
    ax.legend()
    ax.set_xlabel('Time step')
    ax.set_ylabel('Error (deg.)')
    
    return ftl.save_fig(fig, 'p2_tdiff')
    
    
make_fig(x[:-1], x2[:-1])

p2_tdiff.png

## Small Circle

For isolated planets, stable orbits are [small circles](https://en.wikipedia.org/wiki/Circle_of_a_sphere) centered at the planet. Missiles moving along such a small circle have a constant velocity. At a radius of $r=0.2$ ($\approx 11^\circ$) this velocity is

In [19]:
r = .2  # radius of small circle in radians
v_scrcl = grvx.v_scrcl(r)
f'{v_scrcl / grvx.v_esc * 100:.0f}%'

'33%'

of the escape velocity. Time to shoot a missile!

In [20]:
planets = grvx.new_planets([(0., 0.),])
m.set(pos=(r, 0.), orientation=(0., 1.), v=v_scrcl)
x, v = shoot(m, n=20)
x.shape, v.shape

((2000, 3), (2000, 3))

... and to visualize the trajectory

In [21]:
@ftl.with_context
def make_fig(x):
    lat = np.arcsin(x[:, 2])
    lon = np.arctan2(x[:, 0], x[:, 1])

    fig, ax = plt.subplots()
    ax.grid()
    ax.set_xlabel('Longitude (deg.)')
    ax.set_ylabel('Latitude (deg.)')
    ax.plot(lat, lon)
    
    ax.plot(0., 0., 'o', c='black', alpha=.5)
    
    return ftl.save_fig(fig, 'scrcl')


make_fig(x)

scrcl.png

Since we know the expected radius and velocity (GT) we can investigate the residuals:

In [22]:
@ftl.with_context
def make_fig(x, v, r_gt, v_gt):
    lat = np.arcsin(x[:, 2])
    lon = np.arctan2(x[:, 0], x[:, 1])
    dist = np.arccos(np.cos(lat) * np.cos(lon))
    v_abs = np.sqrt(np.sum(v**2, axis=1))
    
    r_error = np.rad2deg(dist - r_gt)
    v_error = np.rad2deg(v_abs - v_gt)

    fig, ax = plt.subplots()
    ax.plot(r_error, label='$r-$GT')
    ax.plot(v_error, label='$v-$GT')

    ax.grid()
    ax.set_xlabel('Time step')
    ax.set_ylabel('Error (deg.)')
    ax.legend()

    return ftl.save_fig(fig, 'scrcl_error')


make_fig(x, v, r, v_scrcl)

scrcl_error.png